In [ ]:
import Pkg

Pkg.add("Interpolations")
Pkg.add("PyPlot")
Pkg.add("PyCall")
Pkg.add("OrdinaryDiffEq")
Pkg.add("YAML")
Pkg.add("DelimitedFiles")
Pkg.add("CSV")
Pkg.add("HDF5")
Pkg.add("StructArrays")
Pkg.add("Random")
Pkg.add("Statistics")

using Interpolations, PyCall, OrdinaryDiffEq,
YAML, DelimitedFiles, CSV, HDF5, StructArrays, Random, 
NBInclude, PyPlot, Statistics

ct = pyimport("cantera")

In [14]:
### COMPUTE u AND T AND χ

struct gas_type
    gas
end

struct AmbientConditions
    "Ambient velocity [m/s]"
    u_a::AbstractFloat
    "Ambient temperature [K]"
    T_a::AbstractFloat
    "Prandtl number"
    Pr::AbstractFloat
    "Lewis number"
    Le::AbstractFloat
    "Specific gas constant of air [J/kg]"
    R::AbstractFloat
    "Ambient pressure [Pa]"
    p::AbstractFloat
    "Initial temperature [K]"
    T0::AbstractFloat
    "Initial velocity [m/s]"
    u0::AbstractFloat
end

### COMPUTE u AND T: FUNCTIONS ###

function compute_y(u::AbstractVector, ρ_rocket::AbstractVector, 
    Δψ::AbstractVector)
    y = ((2 * cumsum((1 ./ (ρ_rocket .* u) .* Δψ)) .^ 0.5))
    return y
end

function solve_exhaust_flow(u_init::AbstractVector, T_init::AbstractVector,
    ambient::AmbientConditions, n::Integer, Δϕ::AbstractVector,
    Δψ::AbstractVector)

    
    u_mem = zeros((size(u_init)[1], n)) # create arrays of size (s,n) to store results 
    T_mem = zeros((size(u_init)[1], n)) # create arrays of size (s,n) to store results
    y_mem = zeros((size(u_init)[1], n)) # create arrays of size (s,n) to store results
    u_mem[:, 1] = u_init # populate u and T result arrays with initial values for ϕ = 0 
    T_mem[:, 1] = T_init # populate u and T result arrays with initial values for ϕ = 0
    ϵ = zeros(n) # turbulent diffusion coefficient array initalization

    for i = 1:n-1 # index to n-1 as we compute n+1
        
        y_mem[:, i] = compute_y(@view(u_mem[:, i]), @view(T_mem[:, i]), Δψ, ambient.R, ambient.p)

        A = construct_tridiagonal_matrix(size(u_init)[1], Δψ, Δϕ[i], @view(y_mem[:, i]), ambient)
        b = construct_rhs(@view(u_mem[:, i]), @view(T_mem[:, i]), @view(y_mem[:, i]), Δψ, Δϕ[i], ambient)
        sol = A \ b
        u_mem[:, i+1] = sol[1:size(u_init)[1]]
        T_mem[:, i+1] = sol[size(u_init)[1]+1:end]
        
        ϵ[i] = get_ϵ(0.02, @view(u_mem[:, i]), @view(y_mem[:, i]))
    end

    y_mem[:, n] = y_mem[:, n-1]

    x = compute_x(ϵ, Δϕ)
    
    return x, y_mem, u_mem, T_mem, ϵ
end

ErrorException: invalid redefinition of constant AmbientConditions

In [17]:
### COMPUTE u AND T ###
# No chemistry

## Constants/Inputs ##
h = [16000] # altitude to caluclate plume properties at [m]
h_string = "16000"
n = 100 # steps in x/ϕ direction
s = 200 # steps in y/ψ direction
m = 1 # counter for test set to only test single altitude
Pr = 1.0 # prandtl number
Le = 1.0 # lewis number
n_species = 53 # number of species in mechanism file

# create spacing arrays for ϕ and ψ
Δϕ = 0.1 * ones(n) #step size in phi
for i = 2:n
    Δϕ[i] = 1.05 * Δϕ[i-1] #enlarge with each step by 1.05
end

Δψ = 0.1 * ones(s) #s vertical grid points
for i = 2:s
    Δψ[i] = 1.05 * Δψ[i-1] #enlarge with each step by 1.05
end

## Initialize loop over altitudes of interest

for m = 1:length(h)
    h_string = string(h[m])

    # Import rocket exit conditions from upstream (u, T, p)
    u0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/u") # initial plume velocity [m/s]
    u0 = convert(AbstractFloat, u0[2])
    T0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T") # initial plume temperature [K]
    T0 = convert(AbstractFloat, T0[2])
    p_all = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/P") #ambient pressure [Pa]
    p = convert(AbstractFloat, p_all[2])
    χ0_full = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/X")
    χ0 = zeros(length(χ0_full[:,2]))

    for d = 1: length(χ0_full[:,2])
        χ0[d] = χ0_full[d,2] # save PPM of species exiting rocket plume
    end

    #curve fit #a = ambient vel [m/s] (speed of rocket) #NEED TO UPDATE W HORIZONTAL VELOCITY
    u_a = 1.11849E-19 * big(h[m])^5 - 1.14814E-14 * big(h[m])^4 + 4.22542E-10 * big(h[m])^3 - 6.92322E-06 * big(h[m])^2 + 6.58761E-02 * big(h[m]) + 5.37920E+01
    T_a = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T_a") # ambient atmospheric temperature [K]
    χ_a = zeros(n_species)
    # set each individual species to respective PPM for ambient air, assume air composition does not change significantly with altitude
    χ_a[48] = 780790 #ppm N2
    χ_a[4] = 209445 #ppm O2
    χ_a[49] = 0 #ppm Ar #CHANGE BACK AFTER DEBUG TO 9339
    χ_a[16] = 9765 #ppm CO2 #426

    #CHECK THAT IMPORTED CONDITIONS ARE CORRECT

    ## Initialize initial u and T conditions: at x/ϕ = 0, y/ψ = all
    gas_init = ct.Solution("gri30.yaml") # create a gas object tuple to define the density with Cantera
    gas_init.TPX = T0, p, χ0 # set state of gas to be rocket exhaust conditions

    u_init = u0 .* ones(s) # array of length y/ψ filled with rocket exit velocity value [m/s]
    T_init = T0 .* ones(s) # array of length y/ψ filled with rocket exit temperature value [K]

    ρ_rocket = ones(s)*gas_init.density_mass # NOTE not the best way but cant figure out circular logic btw density and y NOTE
    R_rocket = p/(ρ_rocket[1]*T0) # NOTE uses ONE R even though it is a feild... how to make individual? NOTE
    
    # we need to define the initial plume geometry by filling the initial u and T arrays with ambient u and T values outside of the plume radius
    radius = 1.147 # rocket exit plume radius [m]

    # have to map ψ in y domain to map properties to the correct grid-points (inside or outside plume radius)
    y_init = compute_y(u_init, ρ_rocket, Δψ) #REDERIVE

    # if y is greater than the radius of the plume, assign the ambient conditions to that point
    u_init[y_init.>radius] .= convert(AbstractFloat, u_a)
    T_init[y_init.>radius] .= convert(AbstractFloat, T_a)

    # Save ambient confditions in struct fo later use
    ambient = AmbientConditions(u_a, T_a, 1.0, 1.0, R_rocket, p, T0, u0)

    #Solve for T and u at all steps: NO CHEMISTRY
    x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)
    
    # save resulting x, y, T, and u 
    x_save = x
    y_save = y
    T_save[m, :, :] = T
    u_save[m, :, :] = u

end

#R = 
#
#ambient = AmbientConditions(u_a, T_a, 1.0, 1.0, R, p, T0, u0)
#
##Solve for T and u at all steps: NO CHEMISTRY
#x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)
#x_save = x
#y_save = y
#T_save[m, :, :] = T
#u_save[m, :, :] = u

MethodError: MethodError: no method matching compute_y(::SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}, ::SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}, ::Array{Float64,1}, ::Float64, ::Float64)
Closest candidates are:
  compute_y(::AbstractArray{T,1} where T, ::AbstractArray{T,1} where T, ::AbstractArray{T,1} where T) at /home/chinahg/GCresearch/rocketemissions/scratch.ipynb:28

In [ ]:
# Computing u and T independent of chemistry
x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)